In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

W0621 13:31:47.708947 139991703807872 deprecation.py:323] From <ipython-input-2-bc596981f0bb>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0621 13:31:47.710846 139991703807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0621 13:31:47.720502 139991703807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0621 13:31:48.578953 139991703807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0621 13:31:48.581532 139991703807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0621 13:31:48.922796 139991703807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
ip_wdth=28
ip_ht=28
ip_channels=1
ip_pixels=784

n_cn1=32
n_cn2=64

s_cn1=1   #stride
s_cn2=1

k_cn1=5
k_cn2=5
k_max_pool1=2
k_max_pool2=2

n_hid=1024
n_out=10

ip_size_to_hidden=(ip_wdth//(k_max_pool1*k_max_pool2))*(ip_ht//(k_max_pool1*k_max_pool2))*n_cn2

In [0]:
wts={
    'wc1':tf.Variable(tf.random_normal([k_cn1,k_cn1,ip_channels,n_cn1])),
    'wc2':tf.Variable(tf.random_normal([k_cn2,k_cn2,n_cn1,n_cn2])),
    'wh1':tf.Variable(tf.random_normal([ip_size_to_hidden,n_hid])),
    'out':tf.Variable(tf.random_normal([n_hid,n_out]))
 }
bias={
    'bc1':tf.Variable(tf.random_normal([n_cn1])),
    'bc2':tf.Variable(tf.random_normal([n_cn2])),
    'bh1':tf.Variable(tf.random_normal([n_hid])),
    'bout':tf.Variable(tf.random_normal([n_out]))

}

In [0]:
def conv(x,wt,bias,strides=1):
  out=tf.nn.conv2d(x,wt,padding='SAME',strides=[1,strides,strides,1])
  
  #strides=[1,strides,strides,1]
  #the strides require a list whose shape should be [no. of images,height,width,no. of channels]
  #we keep no. of images to be 1 because we want to apply filter on each image individually 
  #we don't want to move our filter along the depth so no. of channels s also kept to be 1
  
  out=tf.nn.bias_add(out,bias)
  out=tf.nn.relu(out)
  return out


def maxpooling(x,k=2):
  return tf.nn.max_pool(x,padding='SAME',ksize=[1,k,k,1],strides=[1,k,k,1])


In [0]:
def cnn(x,wts,bias):
  
  x=tf.reshape(x,shape=[-1,ip_ht,ip_wdth,ip_channels])
  
  out_cn1=conv(x,wts['wc1'],bias['bc1'],s_cn1)
  cn1_pool=maxpooling(out_cn1,k_max_pool1)
  
  out_cn2=conv(cn1_pool,wts['wc2'],bias['bc2'],s_cn2)
  cn2_pool=maxpooling(out_cn2,k_max_pool2)
  
  ip_hidden=tf.reshape(cn2_pool,shape=[-1,ip_size_to_hidden])
  op_hidden_before_activation=tf.add(tf.matmul(ip_hidden,wts['wh1']),bias['bh1'])
  op_hidden=tf.nn.relu(op_hidden_before_activation)
  
  output_final=tf.add(tf.matmul(op_hidden,wts['out']),bias['bout'])
  
  return output_final

In [0]:
x=tf.placeholder('float',[None,ip_pixels])
y=tf.placeholder(tf.int32,[None,n_out])
pred=cnn(x,wts,bias)

In [0]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [0]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [0]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
batch_size=100
for i in range(25):
    num_batches=int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        c,_=sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y})
        total_cost+=c
    print(total_cost)
   

959072.2891408205


In [0]:
predictions=tf.argmax(pred,1)
true_labels=tf.argmax(y,1)
correct_predictions=tf.equal(predictions,true_labels)
predictions,correct_predictions=sess.run([predictions,correct_predictions],feed_dict={x:mnist.test.images,y:mnist.test.labels})

In [0]:
correct_predictions.sum()

9862